# MQTT

MQTT es el protocolo estándar de mensajes para los dispositivos IoT (Internet de las Cosas), es decir, para los dispositivos de domótica. Está diseñado para ser extremadamente eficiente y ligero, haciendo que tengamos que enviar muy poca información, con el objetivo de ahorrar recursos, incluyendo batería si es que tenemos sensores o actuadores funcionando con batería o pilas

Este protocolo nos permitirá publicar mensajes y suscribirnos a dichos mensajes, con el objetivo de tener una comunicación bidireccional. Gracias a esta optimización a la hora de enviar mensajes, no solamente ocuparemos pocos recursos del propio dispositivo, sino también de la red Wi-Fi, Bluetooth, ZigBee o la tecnología inalámbrica que uses, haciendo que funcione todo mucho mejor

[MQTT](https://mqtt.org/) (MQ Telemetry Transport) es un protocolo de mensajería para IoT. El broker más popular de MQTT es [Eclipse Mosquitto](https://mosquitto.org/), el cual además es opensource

![MQTT Brocker](https://www.redeszone.net/app/uploads-redeszone.net/2024/01/mqtt-publish-subscribe.png)

En un primer momento, el sensor de temperatura enviará los nuevos datos. El servidor MQTT que se llama «MQTT Broker» reenviará esta información a otros clientes que se han suscrito a dicho mensaje, estos clientes puede ser un simple móvil, o bien nuestro sistema de domótica como Home Assistant, para posteriormente actuar. Hay que tener en cuenta que el principio de funcionamiento de este protocolo es el siguiente: uno o diferentes clientes se suscriben a un TOPIC, el cual es generado por otro cliente y un dispositivo servidor conocido como Broker, de esta manera se encarga de enviar los datos generados por el cliente al resto de clientes que estén suscritos.

## Instalar mosquitto con docker

Primero creamos en la raspberry las carpetas necesarias mediante

``` bash
mkdir ~/docker/mosquitto
mkdir ~/docker/mosquitto/config
mkdir ~/docker/mosquitto/log
mkdir ~/docker/mosquitto/data
```

Dentro de la carpeta de configuración creamos un archivo de configuración

``` bash
touch ~/docker/mosquitto/config/mosquitto.conf
```

Lo abrimos con nano

``` bash
nano ~/docker/mosquitto/config/mosquitto.conf
```

Y añadimos lo siguiente

``` bash
allow_anonymous false
password_file /mqtt/config/passwordfile.txt
pid_file /var/run/mosquitto.pid
persistence true
persistence_location /mqtt/data/
log_dest file /mqtt/log/mosquitto.log
```

Ahora nos vamos al stack de domótica de portainer, dónde instalamos home assistant y node red, y añadimos el siguiente código

```bash
  mqtt:
    container_name: mosquitto
    image: eclipse-mosquitto:latest
    ports:
      - "1883:1883"
      - "9001:9001"
    volumes:
      - ~/docker/mosquitto/config:/mqtt/config
      - ~/docker/mosquitto/log:/mqtt/log
      - ~/docker/mosquitto/data/:/mqtt/data
    restart: unless-stopped
    network_mode: host
```

Actualizamos el stack

Cuando el contenedor se haya levantado, en container le damos al botón `exec` del contenedor de mosquitto que se ha creado. Dentro elegir `/bin/sh`

Una vez dentro del contenedor hacer

``` bash
cd /mqtt/config/
mosquitto_passwd -c passwordfile.txt <usuario>
```

Cambiar `<usuario>` por un nombre de usuario que queramos

Nos pedirá una contraseña, introducir una y guardarla, ya que será la que necesitemos para más adelante usar zigbee2mqtt. Lo que hemos hecho es configurar una contraseña para el usuario `<usuario>`

## Integración en home assistant

En home assistant nos vamos a `Ajustes`->`Dispositivos y servicios`, darle al botón de `añadir integración`, buscar `MQTT`. Dentro, en `Agente` poner `localhost`, en `Puerto` poner `1883` y le damos a `guardar`